In [1]:
#import pandas to read and manipulate data
import pandas as pd

from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

### Script 1 - Filtering and Making Proxy Dataset

In [2]:
#read in the 'nationwide drugs' datasets from FY 2019 - FY2024(YTD)
data21_24 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/nationwide-drugs-fy21-fy24-jan.csv')
data19_22 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/nationwide-drugs-fy19-fy22.csv')
data20_23 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/nationwide-drugs-fy20-fy23.csv')
#filter the following datasets to only display data from 2019 and 2020
#this will make it much simpler to concatenate the data
data19_22 = data19_22.loc[data19_22['FY'] == 2019]
data20_23 = data20_23.loc[data20_23['FY'] == 2020]

#establish the datasets to be concatenated
frames = [data19_22, data20_23, data21_24]

#concatenate data and filter to only Fentanyl records
data = pd.concat(frames)
data = data.loc[data['Drug Type'] == 'Fentanyl']

area_sums = data.groupby(['Area of Responsibility'])['Sum Qty (lbs)'].sum()
test = area_sums.sort_values(ascending=[0])
area_sums.count()

#read in 'amo drug seizures' datasets from FY 2019 - FY2024(YTD)
amo19_22 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/amo-drug-seizures-fy19-fy22.csv')
amo20_23 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/amo-drug-seizures-fy20-fy23.csv')
amo21_24 = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/amo-drug-seizures-fy21-fy24-jan.csv')

#filter the following datasets to only display data from 2019 and 2020
amo19_22 = amo19_22.loc[amo19_22['FY'] == 2019]
amo20_23 = amo20_23.loc[amo20_23['FY'] == 2020]

#establish the datasets to be concatenated
frames = [amo19_22, amo20_23, amo21_24]

#concatenate data and filter to only Fentanyl records 
amo = pd.concat(frames)
amo = amo.loc[amo['Drug Type'] == 'Fentanyl']

#rename the Branch column to allow for an easy concatenation to the 'nationwide drugs' dataset
amo.rename(columns={'Branch': 'Area of Responsibility'}, inplace=True)

#concateneate nationwide drugs and amo drug seizures datasets
df_frames = [data, amo]
df = pd.concat(df_frames)

#save the concatenated data
# df.to_csv('concat_data', index=False)

#change the float value of fentanyl weight to int to allow for Random Forest forecasting
df['Sum Qty (lbs)'] = df['Sum Qty (lbs)'].astype(int)

#organize the areas where fentanyl were seized by overall weight seized (ascending)
df_area_sums = df.groupby(['Area of Responsibility'])['Sum Qty (lbs)'].sum()
df_test = df_area_sums.sort_values(ascending=[0])

#turn this data into a dataframe
Areas_list_df = pd.DataFrame(df_test)

#create a separate column in this dataframe to order the list, these values will act as proxies
Areas_list_df = Areas_list_df.assign(row_number=range(len(Areas_list_df))) +1

#rename the proxy column
Areas_list_df.rename({'row_number': 'Area Proxy'}, axis=1, inplace=True)

#save the areas and proxy variables for data documentation
# Areas_list_df.to_csv('Areas_list_proxy', index=True)
# Above is stored at: 'https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/Areas_list_proxy'

#merge the proxy list with the concatenated data file
proxy_df = pd.merge(df, Areas_list_df, on=["Area of Responsibility"])

#delete the column for overall fentanyl weight by area
proxy_df = proxy_df.drop('Sum Qty (lbs)_y', axis=1)

#assign integer values to the FY column which was previously a string
FY_map = {'2019': 2019, '2020': 2020, '2021': 2021, '2022': 2022, '2023': 2023, '2024 (FYTD)': 2024}
proxy_df = proxy_df.replace({'FY': FY_map})

#rename columns 
proxy_df = proxy_df.rename(columns={'Sum Qty (lbs)_x': 'Weight', 'Count of Event': 'Count'})

#save the new proxy dataset
# proxy_df.to_csv('proxy_df', index=False)
# Above is stored at: 'https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/proxy_df'

### Script 2

In [3]:
#load in the previously created proxy dataset
df = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/proxy_df')

#I want to add a column and proxy column for only cities, so I am getting rid of all the extra column values
df['City'] = df['Area of Responsibility'].str.replace(r' FIELD OFFICE', '', regex=True)
df['City'] = df['City'].str.replace(r' SECTOR', '', regex=True)
df['City'] = df['City'].str.replace(r' Air And Marine Branch', '', regex=True)
df['City'] = df['City'].str.replace(r'NASOC - ', '', regex=True)
df['City'] = df['City'].str.replace(r' IAC', '', regex=True)
df['City'] = df['City'].str.replace(r'AMO Operation ', '', regex=True)
df['City'] = df['City'].str.replace(r'National Air Training Center ', '', regex=True)
df['City'] = df['City'].str.replace(r' Air Branch', '', regex=True)
df['City'] = df['City'].str.replace(r'National Air Training Center ', '', regex=True)

#convert all cities to uppercase so they merge without a manual join
df['City'] = df['City'].str.upper()

#isolate the cities and group them by weight, then turn this into a dataframe
city_sums = df.groupby(['City'])['Weight'].sum()
city_df = pd.DataFrame(city_sums.sort_values(ascending=[0]))

#assign numbers to each column variable as proxies, then rename the column
city_df = city_df.assign(row_number=range(len(city_df))) +1
city_df.rename({'row_number': 'City Proxy'}, axis=1, inplace=True)

#merge the city proxy dataframe with the larger dataset
city_proxy_df = pd.merge(df, city_df, on=["City"])

#remove the column with grouped sums for each city
city_proxy_df = city_proxy_df.drop('Weight_y', axis=1)


# city_df.to_csv('city_list_proxy', index=True)
# Above is stored at: 'https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/city_list_proxy'

#add columns which which will convert the remaining non-numeric variables into proxies
city_proxy_df['Month_proxy'] = city_proxy_df['Month (abbv)']
city_proxy_df['Component_proxy'] = city_proxy_df['Component']
city_proxy_df['Region_proxy'] = city_proxy_df['Region']
city_proxy_df['Land_proxy'] = city_proxy_df['Land Filter']
city_proxy_df['Drug_proxy'] = city_proxy_df['Drug Type']

#create maps with new variables for each column variable
month_map = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06', 
             'JUL':'07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}
component_map = {'Office of Field Operations': 1, 'U.S. Border Patrol': 2,
       'Air and Marine Operations': 3}
region_map = {'Coastal/Interior': 1, 'Northern Border': 2, 'Southwest Border': 3}
land_map = {'Land': 1, 'Land Only': 2, 'Other': 3}
drug_map = {'Fentanyl': 1}

#map the new proxy variables to new columns
city_proxy_df = city_proxy_df.replace({'Month_proxy': month_map, 'Component_proxy': component_map, 
                                       'Region_proxy': region_map, 'Land_proxy': land_map, 
                                       'Drug_proxy': drug_map})

#create column for a specific datetime
city_proxy_df['FY_date'] = city_proxy_df['FY'].astype(str) + '-' + city_proxy_df['Month_proxy'].astype(str)
city_proxy_df['FY_date'] = pd.to_datetime(city_proxy_df['FY_date'], format='%Y-%m')

city_proxy_df.head()

#save the new data with all proxy variables
# city_proxy_df.to_csv('all_proxy_data', index=False)
# Above is stored at: https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/all_proxy_data

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count,Weight_x,Area Proxy,City,City Proxy,Month_proxy,Component_proxy,Region_proxy,Land_proxy,Drug_proxy,FY_date
0,2019,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Fentanyl,1,2,25,BALTIMORE,19,04,1,1,3,1,2019-04-01
1,2019,AUG,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Fentanyl,1,0,25,BALTIMORE,19,08,1,1,3,1,2019-08-01
2,2020,AUG,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Fentanyl,2,4,25,BALTIMORE,19,08,1,1,3,1,2020-08-01
3,2020,JUN,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Fentanyl,1,4,25,BALTIMORE,19,06,1,1,3,1,2020-06-01
4,2020,MAY,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Fentanyl,1,0,25,BALTIMORE,19,05,1,1,3,1,2020-05-01


### Script 3 - Implement RF

In [4]:
#load previously created proxy data file
df = pd.read_csv('https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/all_proxy_data')

#convert FY_date to a datetime
df['FY_date'] = pd.to_datetime(df['FY_date'])

#convert FY_date to an ordinal date
df['FY_date_ordinal'] = df['FY_date'].map(pd.Timestamp.toordinal)

#rename columns
df = df.rename(columns={'Weight_x': 'Weight', 'City Proxy': 'City_proxy', 
                        'Area Proxy': 'Area_proxy'})

#filter the large dataframe (proxy_data) to only display the numerical values
df2 = df.filter(['FY','FY_date', 'FY_date_ordinal','Month_proxy', 'Region_proxy', 
                 'Component_proxy','Land_proxy', 'City_proxy', 'Area_proxy', 
                 'Drug_proxy', 'Count', 'Weight'], axis=1)

#save the numerical data file to be used for Random Forest forecasting
# df2.to_csv('RF_data', index=False)
# Above is stored at:  https://raw.githubusercontent.com/claytondmalone/Business-Forecasting/main/RF_data

#filter the dataframe to just display results from region 2
#this can be done for any of the 3 regions for analysis
df3 = df2.loc[df2['Region_proxy'] == 2]

#set x and y variables
x = df3.drop(['FY_date','Weight', 'Region_proxy'], axis=1)
y = df3['Weight']

# Split the data into train and test sets
x, xt, y, yt = train_test_split(x, y, test_size=0.2)

# Create the model and fit it using the train data
clf = RF(n_estimators=100, n_jobs=-1, max_depth=5)
clf.fit(x, y)

pred = clf.predict(xt)
acc = accuracy_score(yt, pred)

print("Model accuracy is {}%.".format(acc*100))

Model accuracy is 62.71186440677966%.
